In [2]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import re, glob
import os, sys
from scipy import stats

In [3]:
country_tracks_data_df = pd.read_csv("Brazil_FIX_tracks_with_audio_features.csv")

country_tracks_data_df.head()

,Position,Track Name,Artist,Streams,URL,Year,Month,Track ID,Danceability,Energy,Valence
0,1,Atrasadinha - Ao Vivo,Felipe Araújo,4193494,https://open.spotify.com/track/5EKHbW7mXcAR4hC...,2019,1,3tL7UqLFy2XcYW1AAA60RV,0.676,0.844,0.716
1,2,Jenifer,Gabriel Diniz,3640286,https://open.spotify.com/track/5ixqGGFu0hdHfEr...,2019,1,7kh64k3P9Fk4EsA6vOdwmj,0.719,0.825,0.798
2,3,Notificação Preferida - Ao Vivo,Zé Neto & Cristiano,3600348,https://open.spotify.com/track/2qkNnJvQvhLiZ63...,2019,1,3tA0vALUwrzkTgHcm9j6oJ,0.398,0.838,0.206
3,4,Agora é Tudo Meu,DENNIS,3492545,https://open.spotify.com/track/5yH8pYxNckOU1cx...,2019,1,5yH8pYxNckOU1cxfPsMIaz,0.848,0.933,0.506
4,5,Quem Me Dera,Márcia Fellipe,3386497,https://open.spotify.com/track/2lAyctg5FocS67e...,2019,1,2lAyctg5FocS67eksHoyIg,0.815,0.723,0.896


In [30]:
#country_tracks_data_df = country_tracks_data_df.rename(columns={"Energy": "Energy2", 'Valence': "Energy"})
#country_tracks_data_df.head()

In [31]:
#country_tracks_data_df = country_tracks_data_df.rename(columns={"Energy2": "Valence"})
#country_tracks_data_df.head()

In [4]:
#Tot_Streams = country_tracks_data_df['Streams'].sum() 
tot_streams_groupby = country_tracks_data_df.groupby(country_tracks_data_df['Year', 'Month'])
tot_streams_groupby


#country_tracks_data_df['Stream %'] = country_tracks_data_df['Streams'] / Tot_Streams
#country_tracks_data_df.head()

KeyError: ('Year', 'Month')

In [33]:
country_tracks_data_df['Danceability_Stream%'] = country_tracks_data_df['Danceability'] * country_tracks_data_df['Stream %']    
country_tracks_data_df['Valence_Stream%'] = country_tracks_data_df['Valence'] * country_tracks_data_df['Stream %'] 
country_tracks_data_df['Energy_Stream%'] = country_tracks_data_df['Energy'] * country_tracks_data_df['Stream %'] 
country_tracks_data_df.head()

,Position,Track Name,Artist,Streams,URL,Year,Month,Track ID,Danceability,Energy,Valence,Stream %,Danceability_Stream%,Valence_Stream%,Energy_Stream%
0,1,7 rings,Ariana Grande,71486,https://open.spotify.com/track/6ocbgoVGwYJhOv1...,2019,1,6ocbgoVGwYJhOv1GgI9NsF,0.778,0.317,0.327,0.000066,0.000051,0.000022,0.000021
1,2,Shallow,Lady Gaga,67267,https://open.spotify.com/track/2VxeLyX666F8uXC...,2019,1,2VxeLyX666F8uXCJ0dZF8B,0.572,0.385,0.323,0.000062,0.000036,0.000020,0.000024
2,3,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,65006,https://open.spotify.com/track/7dbka99KTWke5o9...,2019,1,3KkXRkHbMCARz0aVfEt68P,0.760,0.479,0.913,0.000060,0.000046,0.000055,0.000029
3,4,"break up with your girlfriend, i'm bored",Ariana Grande,62512,https://open.spotify.com/track/4kV4N9D1iKVxx1K...,2019,1,4kV4N9D1iKVxx1KLvtTpjS,0.726,0.554,0.335,0.000058,0.000042,0.000019,0.000032
4,5,Con Calma,Daddy Yankee,59178,https://open.spotify.com/track/5w9c2J52mkdntKO...,2019,1,5w9c2J52mkdntKOmRLeM2m,0.737,0.860,0.656,0.000055,0.000040,0.000036,0.000047


In [34]:
country_tracks_data_df.to_csv("India_tracks_with_audio_features.csv", index=False)